In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Load the dataset
file_path = "S&P500_L5Y.csv"  # Update this if needed
df = pd.read_csv(file_path)

# Convert Date column to datetime format and sort in ascending order
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y')
df = df.sort_values(by='Date')

# Rename 'Close/Last' for easier reference
df.rename(columns={'Close/Last': 'Close'}, inplace=True)

# Compute log returns
df['Log_Returns'] = np.log(df['Close'] / df['Close'].shift(1))

# Drop first row due to NaN in log returns
df = df.dropna()

# Perform Augmented Dickey-Fuller test
adf_test = adfuller(df['Log_Returns'])
print("ADF Statistic:", adf_test[0])
print("p-value:", adf_test[1])
print("Critical Values:", adf_test[4])
print("Is the series stationary?", adf_test[1] < 0.05)  # Null hypothesis: non-stationary

# Split into training (256 weeks) and testing (4 weeks)
train_size = 256 * 5  # 1280 days
test_size = 4 * 5  # 20 days

train_data = df.iloc[:-test_size]  # All but the last 20 days
test_data = df.iloc[-test_size:]  # Last 20 days

# Save to CSV if needed
train_data.to_csv("train_data.csv", index=False)
test_data.to_csv("test_data.csv", index=False)

print("Training data shape:", train_data.shape)
print("Testing data shape:", test_data.shape)


ADF Statistic: -43.45092138266382
p-value: 0.0
Critical Values: {'1%': np.float64(-3.4355754676859886), '5%': np.float64(-2.8638475772391665), '10%': np.float64(-2.5679985805677017)}
Is the series stationary? True
Training data shape: (1235, 6)
Testing data shape: (20, 6)


In [ ]:

# Load preprocessed data
df = pd.read_csv("train_data.csv")  # Load the training dataset
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

# Plot ACF and PACF to determine p and q
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
plot_acf(df['Log_Returns'], ax=axes[0])  # Identify q
plot_pacf(df['Log_Returns'], ax=axes[1])  # Identify p
plt.show()

# Fit ARIMA model (use d=0 since log returns are already stationary)
p, d, q = 1, 0, 1  # Adjust based on ACF/PACF analysis
model = ARIMA(df['Log_Returns'], order=(p, d, q))
model_fit = model.fit()

# Print model summary
print(model_fit.summary())

# Forecast future log returns for the test period
test_df = pd.read_csv("test_data.csv")
test_df['Date'] = pd.to_datetime(test_df['Date'])
test_df.set_index('Date', inplace=True)

forecast = model_fit.forecast(steps=len(test_df))

# Evaluate the model
rmse = np.sqrt(mean_squared_error(test_df['Log_Returns'], forecast))
mae = mean_absolute_error(test_df['Log_Returns'], forecast)

print(f"RMSE: {rmse:.5f}")
print(f"MAE: {mae:.5f}")

# Plot actual vs. predicted
plt.figure(figsize=(10,5))
plt.plot(test_df.index, test_df['Log_Returns'], label="Actual Log Returns", marker='o')
plt.plot(test_df.index, forecast, label="Predicted Log Returns", linestyle='dashed', marker='x')
plt.legend()
plt.title("ARIMA Forecast vs Actual")
plt.show()
